In [2]:
import pandas as pd
from beanie import init_beanie, PydanticObjectId
from motor.motor_asyncio import AsyncIOMotorClient
from app.models import models
import app.utils.pandas_utils as pu
from app.models.researchers import Researcher
from app.models.works import Work

In [3]:
client = AsyncIOMotorClient("mongodb://localhost:27017")
await init_beanie(database=client["nexus_dev"], document_models=models)

In [17]:
works = await Work.find(Work.authors.id == PydanticObjectId("675729251fc3e5ed73dd6f53")).to_list()
df = pu.read_from_beanie(works).set_index("id")

In [18]:
df

,uuid,imported_at,manually_updated_at,external_id,title,type,publication_year,publication_date,keywords,authors,language,open_access,openalex_meta,orcid_meta,dblp_meta
id,,,,,,,,,,,,,,,
675729261fc3e5ed73dd6fd9,8d611a7e-cea6-42f4-8830-dcefc058bf22,2024-12-09 18:30:14.151,None,"{'openalex': None, 'orcid': None, 'dblp': '268...",Model-driven engineering support for social an...,"{'openalex': None, 'orcid': None, 'dblp': 'Con...",2019,None,None,[<beanie.odm.fields.Link object at 0x10cc7f410...,None,False,None,None,"{'@score': '3', '@id': '2688909', 'info': {'au..."
6757292a1fc3e5ed73dd717d,40097f96-8f8d-4ae1-934b-903ef4cc8065,2024-12-09 18:30:17.507,None,"{'openalex': None, 'orcid': None, 'dblp': '290...",Considerations about quality in model-driven e...,"{'openalex': None, 'orcid': None, 'dblp': 'Jou...",2018,None,None,[<beanie.odm.fields.Link object at 0x10cbe5e90...,None,False,None,None,"{'@score': '3', '@id': '2906511', 'info': {'au..."


In [29]:
works = await Work.find({"authors.full_name": "Federico Ciccozzi"}, fetch_links=True).to_list()
df = pu.read_from_beanie(works)

In [30]:
df

,id,uuid,imported_at,manually_updated_at,external_id,title,type,publication_year,publication_date,keywords,authors,language,open_access,openalex_meta,orcid_meta,dblp_meta
0,675729261fc3e5ed73dd6ff0,19a810bd-2c8c-4505-abbd-0adb51533e35,2024-12-09 18:30:14.889,None,"{'openalex': 'W4405124417', 'orcid': None, 'db...",An elucidation of blended modeling from an ind...,"{'openalex': 'article', 'orcid': None, 'dblp':...",2024,2024-12-06,"[Model-driven architecture, Modeling language]","[{'id': 675729261fc3e5ed73dd6f98, 'uuid': cc5c...",en,True,"{'id': 'https://openalex.org/W4405124417', 'do...",None,None
1,675729b61fc3e5ed73dd73f8,3e60a486-2323-42b2-9e59-04feb989f088,2024-12-09 18:32:37.464,None,"{'openalex': None, 'orcid': None, 'dblp': '326...",Model-Driven Engineering for Mission-Critical ...,"{'openalex': None, 'orcid': None, 'dblp': 'Jou...",2017,None,None,"[{'id': 675729291fc3e5ed73dd710c, 'uuid': 14df...",None,False,None,None,"{'@score': '3', '@id': '3269491', 'info': {'au..."


In [27]:
researchers = await Researcher.aggregate([
    {"$group" : {"_id":"$full_name", "count":{"$sum":1}}},
    {"$match": {"count": {"$gt": 1}}}
]).to_list()
researchers = pd.DataFrame(researchers)

In [28]:
researchers

,_id,count
0,Jun Sun,2
1,Federico Ciccozzi,2
2,Holger Giese,2
3,Manuel Wimmer,2
